In [1]:
# ====================================================================
# [ 1. CONFIGURAZIONE ]
# ====================================================================

class Config:
    # Percorsi file
    SMILES_FILE = "/home/grad/Desktop/pietro/denovo/2/compacted_smiles.txt"
    SMARTS_RX_FILE = "/home/grad/Desktop/pietro/denovo/2/SMART_RX/smartsrx-main/smartsrx.json"
    
    VOCAB_PATH = "/home/grad/Desktop/pietro/denovo/2/smartrx/vocab.json"
    # Iperparametri Modello
    EMBED_DIM = 512
    TRANSFORMER_LAYERS = 6
    TRANSFORMER_HEADS = 6
    FF_DIM = 2048
    DROPOUT_RATE = 0.10
    L2_REG = 1e-4
    
    # Training
    BATCH_SIZE = 64
    EPOCHS = 400
    MAX_LENGTH = 100         # Adatto per SMARTS + SMILES
    GRADIENT_CLIP = 1.0
    VALID_RATIO = 0.1
    
    # Curriculum & Augmentation
    CURRICULUM_START_COMPLEXITY = 10
    CURRICULUM_COMPLEXITY_STEP = 5
    LOSS_STABILITY_THRESHOLD = 0.01
    WARMUP_EPOCHS = 5
    AUGMENT_PROB = 0.10
    
    # Generazione
    TEMPERATURE = 1.0
    GEN_NUM = 10
    PRINT_EVERY = 100
    STEPS_PER_EPOCH = 1000

config = Config()



In [ ]:
# ============================================================================
# TRAINING: SMARTS-RX → SMILES (NO SCAFFOLD)
# ============================================================================
import os
import sys
import time
import random
import re
import logging
import json
import pickle
import numpy as np
import tensorflow as tf
from threading import Lock
from typing import List, Tuple, Optional

# RDKit Imports
from rdkit import Chem
from rdkit.Chem import MolFromSmiles, MolToSmiles
from rdkit.Chem.FilterCatalog import FilterCatalog, FilterCatalogEntry, SmartsMatcher 
from rdkit.Chem import FilterCatalog  # Necessario per SMARTS-RX
from sklearn.model_selection import train_test_split

# TensorFlow Imports
from tensorflow.keras.layers import Layer, Embedding, Input, LayerNormalization, MultiHeadAttention, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import Callback, ModelCheckpoint

# ============================================================================
# TRAINING: SMARTS-RX → SMILES (SENZA SCAFFOLD)
# ============================================================================
import os
import sys
import time
import random
import re
import logging
import json
import pickle
import numpy as np
import tensorflow as tf
from threading import Lock
from typing import List, Tuple, Optional

# RDKit Imports
from rdkit import Chem
from rdkit.Chem import MolFromSmiles, MolToSmiles
from rdkit.Chem.FilterCatalog import FilterCatalog, FilterCatalogEntry, SmartsMatcher
from sklearn.model_selection import train_test_split

# TensorFlow Importsnew/cam
from tensorflow.keras.layers import Layer, Embedding, Input, LayerNormalization, MultiHeadAttention, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import Callback, ModelCheckpoint

# Configurazione Logger
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")
# ====================================================================
# [ 2. GESTIONE SMARTS-RX ]
# ====================================================================

SMARTS_CATALOG = None

def initialize_smarts_catalog():
    """Carica il catalogo SMARTS-RX."""
    global SMARTS_CATALOG
    if SMARTS_CATALOG is not None: return

    try:
        with open(config.SMARTS_RX_FILE, "rt", encoding="utf-8") as f:
            data = json.load(f)
    except Exception as e:
        logger.error(f"Errore caricamento SMARTS JSON: {e}")
        sys.exit(1)

    catalog = FilterCatalog()
    count = 0
    for entry in data.get("data", []):
        name = entry.get("specific_type")
        smarts = entry.get("smarts")
        if name and smarts:
            pattern = Chem.MolFromSmarts(smarts)
            if pattern:
                catalog.AddEntry(FilterCatalogEntry(name, SmartsMatcher(pattern)))
                count += 1
    SMARTS_CATALOG = catalog
    logger.info(f"✓ Catalogo SMARTS-RX caricato: {count} regole.")

def get_smarts_fingerprint(mol: Chem.Mol) -> List[str]:
    """Restituisce lista nomi gruppi funzionali presenti."""
    if not mol or SMARTS_CATALOG is None: return []
    matches = SMARTS_CATALOG.GetMatches(mol)
    return sorted(list(set([m.GetDescription() for m in matches])))

# ====================================================================
# [ 3. HELPER CHIMICI ]
# ====================================================================
def robust_tokenize(smiles: str) -> list:
    # PRIORITÀ ASSOLUTA: Atomi aromatici (c, n, o) prima delle maiuscole
    pattern = (
        r"(\[[^\[\]]{1,6}\]|"                 # 1. Atomi tra parentesi [C@@H]
        r"Br|Cl|Si|Na|Mg|Mn|Ca|Fe|Zn|Se|Li|K|Al|B|"  # 2. Elementi specifici (2 lettere)
        r"c|n|o|s|p|"                         # 3. 🚨 FIX: Atomi aromatici (1 lettera minuscola)
        r"R[0-9]|r[0-9]|a[0-9]|"              # 4. Ring labels
        r"[A-Za-z0-9@+\-\\\/\(\)=#\$\.\%,])"  # 5. Tutto il resto
    )
    
    tokens = re.findall(pattern, smiles)
    
    # Controllo parentesi (stack) semplificato
    stack = []
    for t in tokens:
        if t.startswith('['): stack.append(t)
        if t.endswith(']'):
            if not stack: return [] # Parentesi chiusa senza aperta
            stack.pop()
    
    # Se lo stack non è vuoto, ci sono parentesi non chiuse
    if stack: return []
    
    # Check finale: se unendo i token non esce una molecola valida, scarta
    try:
        if not Chem.MolFromSmiles(''.join(tokens)):
            return []
    except:
        return []
        
    return tokens

def validate_and_fix_smiles(smiles: str) -> str:
    try:
        mol = Chem.MolFromSmiles(smiles, sanitize=True)
        if mol is None: return None
        try: Chem.Kekulize(mol, clearAromaticFlags=True)
        except: pass
        return Chem.MolToSmiles(mol, canonical=True, isomericSmiles=False)
    except: return None

def randomize_smiles(smiles: str, num_versions: int = 1) -> List[str]:
    mol = Chem.MolFromSmiles(smiles)
    if not mol: return []
    res = []
    try:
        # FIX: Forziamo la Kekulizzazione per coerenza con il vocabolario
        Chem.Kekulize(mol, clearAromaticFlags=True)
        # Generiamo SMILES random ma mantenendo la forma di Kekulé
        s = Chem.MolToSmiles(mol, doRandom=True, canonical=False, kekuleSmiles=True)
        if s: res.append(s)
    except: pass
    return res

def compute_complexity_from_tokens(tokens: List[str]) -> int:
    smiles = ''.join(tokens)
    try:
        mol = Chem.MolFromSmiles(smiles)
        if not mol: return 999
        return Chem.GetSSSR(mol) + smiles.count('(')
    except: return 999

# ====================================================================
# [ 4. PREPROCESSING DATASET (SMARTS + SMILES) ]
# ====================================================================

def process_dataset(data: List[str]) -> Tuple[List[Tuple[List[str], List[str]]], List[str], int]:
    """
    Output Tuple: (SMARTS_TOKENS, TARGET_TOKENS)
    SMARTS_TOKENS: lista di nomi gruppi funzionali
    TARGET_TOKENS: lista di token SMILES
    """
    initialize_smarts_catalog()
    
    processed = []
    all_tokens = set()
    
    logger.info("→ Inizio Preprocessing (SMARTS-RX + SMILES)...")
    
    for idx, s in enumerate(data):
        if idx % 10000 == 0: logger.info(f"  Processati {idx}/{len(data)}")
            
        fixed = validate_and_fix_smiles(s)
        if not fixed: continue
        
        mol = Chem.MolFromSmiles(fixed)
        if not mol: continue

        # 1. Target (SMILES) → Tokenizza
        tokens_target = robust_tokenize(fixed)
        if not tokens_target: continue
        
        # 2. SMARTS-RX → Lista di nomi (NON tokenizzare!)
        smarts_names = get_smarts_fingerprint(mol)
        
        # STRUTTURA CORRETTA: (['Alcohol', 'Ketone'], ['C', 'C', '=', 'O', ...])
        total_len = 1 + len(smarts_names) + 1 + len(tokens_target) + 1

        if 4 <= total_len <= config.MAX_LENGTH:
            processed.append((smarts_names, tokens_target))  # ✅ CORRETTO
            all_tokens.update(tokens_target)
            all_tokens.update(smarts_names)

    # Costruzione Vocabolario
    vocab = ['<PAD>', '<START>', '<END>', '<SEP>']
    vocab.extend(sorted(list(all_tokens)))
    
    # Ricalcolo Max Len
    lengths = []
    for smarts_tokens, target_tokens in processed:
        lengths.append(1 + len(smarts_tokens) + 1 + len(target_tokens) + 1)
        
    if processed:
        max_len = min(int(np.percentile(lengths, 99)), config.MAX_LENGTH)
    else:
        max_len = config.MAX_LENGTH
    
    logger.info(f"✓ Preprocessing completato: {len(processed)} molecole | Vocab: {len(vocab)} | MaxLen: {max_len}")
    return processed, vocab, max_len

# ====================================================================
# [ 5. CLASSI THREAD-SAFE ]
# ====================================================================

class ThreadSafeIterator:
    def __init__(self, iterator):
        self.iterator = iterator
        self.lock = Lock()
    def __iter__(self): return self
    def __next__(self):
        with self.lock: return next(self.iterator)

def threadsafe_generator(func):
    def wrapper(*args, **kwargs): return ThreadSafeIterator(func(*args, **kwargs))
    return wrapper

# ====================================================================
# [ 6. GENERATORE CURRICULUM (2 PARTI) ]
# ====================================================================

class CurriculumSmilesGenerator:
    """
    Generatore di sequenze per training con curriculum learning.
    Formato: [START] SMARTS [SEP] TARGET [END]
    Nessun token <UNK> - tutti i token devono essere nel vocabolario.
    """
    
    def __init__(self, processed_data: List[Tuple[List[str], List[str]]], vocab: List[str]):
        """
        Args:
            processed_data: Lista di tuple (smart_names, target_tokens)
            vocab: Lista completa di token nel vocabolario
        """
        # Mappature token ↔ indice
        self.char2idx = {c: i for i, c in enumerate(vocab)}
        self.idx2char = {i: c for c, i in self.char2idx.items()}
        
        # Verifica token essenziali
        essential_tokens = ['<PAD>', '<START>', '<END>', '<SEP>']
        for token in essential_tokens:
            if token not in self.char2idx:
                raise ValueError(f"Token essenziale '{token}' mancante nel vocabolario!")
        
        # Dati originali con complessità
        self.original_data = []
        for smarts, target in processed_data:
            comp = compute_complexity_from_tokens(target)
            self.original_data.append(((smarts, target), comp))
        
        # Parametri curriculum
        valid_comps = [c for _, c in self.original_data if c != 999]
        self.max_complexity = max(valid_comps) if valid_comps else 0
        self.current_complexity = config.CURRICULUM_START_COMPLEXITY
        self.available_data = self._filter_data()
        
        # Per novelty check
        self.train_smiles = {''.join(target) for (_, target), _ in self.original_data}
        
        self.lock = Lock()
    
    def _filter_data(self) -> List[Tuple[Tuple[List[str], List[str]], int]]:
        """Filtra i dati per complessità <= current_complexity."""
        filtered = [dp for dp, c in self.original_data if c <= self.current_complexity]
        return filtered if filtered else [dp for dp, _ in self.original_data]
    
    def update_complexity(self, epoch: int, loss_diff: float = None):
        """
        Aggiorna la complessità del curriculum.
        Args:
            epoch: Epoca corrente
            loss_diff: Differenza di loss (se None, usa strategia warmup)
        """
        with self.lock:
            if loss_diff is not None and loss_diff < config.LOSS_STABILITY_THRESHOLD:
                # Aumenta complessità se la loss è stabile
                self.current_complexity = min(
                    self.current_complexity + config.CURRICULUM_COMPLEXITY_STEP, 
                    self.max_complexity
                )
            else:
                # Strategia warmup
                if epoch <= config.WARMUP_EPOCHS:
                    incr = int((self.max_complexity - config.CURRICULUM_START_COMPLEXITY) * 
                              (epoch / config.WARMUP_EPOCHS))
                    self.current_complexity = config.CURRICULUM_START_COMPLEXITY + incr
                else:
                    self.current_complexity = self.max_complexity
            
            self.available_data = self._filter_data()
            if not self.available_data:
                self.available_data = [dp for dp, _ in self.original_data]
                logger.warning("Reset available_data (fallback).")
    
    @threadsafe_generator
    def __call__(self):
        """
        Genera batch di (inputs, targets) per il training.
        Yield: (inputs, targets) dove targets[i] = inputs[i][1:] + <PAD>
        """
        PAD_IDX = self.char2idx['<PAD>']
        START_IDX = self.char2idx['<START>']
        END_IDX = self.char2idx['<END>']
        SEP_IDX = self.char2idx['<SEP>']

        while True:
            inputs = np.full((config.BATCH_SIZE, config.MAX_LENGTH), PAD_IDX, dtype=np.int32)
            targets = np.full_like(inputs, PAD_IDX)
            
            for i in range(config.BATCH_SIZE):
                # Seleziona esempio dal dataset corrente
                with self.lock:
                    try:
                        data_pair = random.choice(self.available_data)
                    except:
                        self.available_data = [dp for dp, _ in self.original_data]
                        data_pair = random.choice(self.available_data)
                
                # Unpacking: (smart_names, target_tokens)
                smarts_names, target_tokens = data_pair

                # Augmentation solo sul target
                curr_target = target_tokens
                if random.random() < config.AUGMENT_PROB:
                    try:
                        aug = randomize_smiles(''.join(target_tokens), 1)
                        if aug:
                            tok = robust_tokenize(aug[0])
                            if tok: curr_target = tok
                    except: pass

                # Costruisci sequenza completa
                # Formato: [START] SMARTS [SEP] TARGET [END]
                seq = (['<START>'] + 
                       smarts_names + 
                       ['<SEP>'] + 
                       curr_target + 
                       ['<END>'])
                
                # Padding
                padded_seq = (seq + ['<PAD>'] * config.MAX_LENGTH)[:config.MAX_LENGTH]
                
                # Converti in indici (tutti i token sono garantiti esistere)
                indices = [self.char2idx[t] for t in padded_seq]
                
                # Assegna input e target
                inputs[i] = indices
                targets[i, :-1] = indices[1:]  # Shift di 1
                targets[i, -1] = PAD_IDX       # Ultima posizione padding
                
            yield inputs, targets

    def get_dataset(self):
        """Restituisce un tf.data.Dataset pronto per il training."""
        return tf.data.Dataset.from_generator(
            self.__call__,
            output_signature=(
                tf.TensorSpec(shape=(config.BATCH_SIZE, config.MAX_LENGTH), dtype=tf.int32),
                tf.TensorSpec(shape=(config.BATCH_SIZE, config.MAX_LENGTH), dtype=tf.int32)
            )
        ).prefetch(tf.data.AUTOTUNE)

# ====================================================================
# [ 7. MODELLO TRANSFORMER ]
# ====================================================================

# 1. DEFINIZIONE LOSS
def smoothed_loss(y_true, y_pred):
    y_true_int = tf.cast(y_true, tf.int32)
    mask = tf.cast(tf.math.not_equal(y_true, 0), tf.float32)
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_true_int, logits=y_pred)
    return tf.reduce_sum(loss * mask) / (tf.reduce_sum(mask) + 1e-9)

# 2. LAYERS CUSTOM
class DynamicPositionalEncoding(Layer):
    def __init__(self, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        
    def build(self, input_shape):
        max_len = config.MAX_LENGTH
        pos = np.arange(max_len)[:, np.newaxis]
        i = np.arange(self.embed_dim)[np.newaxis, :]
        angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(self.embed_dim))
        angle_rads = pos * angle_rates
        angle_rads[:, 0::2] = tf.math.sin(angle_rads[:, 0::2])
        angle_rads[:, 1::2] = tf.math.cos(angle_rads[:, 1::2])
        self.pos_encoding = tf.cast(angle_rads[np.newaxis, ...], dtype=tf.float32)
        
    def call(self, inputs):
        seq_len = tf.shape(inputs)[1]
        return inputs + self.pos_encoding[:, :seq_len, :]
        
    def get_config(self):
        return {**super().get_config(), "embed_dim": self.embed_dim}

class ImprovedTransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ffn_dim, rate=0.1, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.ffn_dim = ffn_dim
        self.rate = rate
        
        self.mha = MultiHeadAttention(
            num_heads=num_heads, 
            key_dim=embed_dim, 
            kernel_regularizer=tf.keras.regularizers.l2(config.L2_REG)
        )
        self.ffn = tf.keras.Sequential([
            Dense(ffn_dim, activation="gelu", kernel_regularizer=tf.keras.regularizers.l2(config.L2_REG)),
            Dense(embed_dim, kernel_regularizer=tf.keras.regularizers.l2(config.L2_REG))
        ])
        self.ln1 = LayerNormalization(epsilon=1e-6)
        self.ln2 = LayerNormalization(epsilon=1e-6)
        self.d1 = Dropout(rate)
        self.d2 = Dropout(rate)

    def call(self, inputs, training=False, mask=None):
        seq_len = tf.shape(inputs)[1]
        causal_mask = tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
        attn_output = self.mha(inputs, inputs, attention_mask=causal_mask)
        out1 = self.ln1(inputs + self.d1(attn_output, training=training))
        ffn_output = self.ffn(out1)
        return self.ln2(out1 + self.d2(ffn_output, training=training))
        
    def get_config(self):
        return {**super().get_config(), "embed_dim": self.embed_dim, "num_heads": self.num_heads, "ffn_dim": self.ffn_dim, "rate": self.rate}

class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, embed_dim, warmup_steps=10000):
        super().__init__()
        # FIX: Usa tf.cast per evitare errori di tipo
        self.embed_dim = tf.cast(embed_dim, tf.float32)
        self.warmup_steps = tf.cast(warmup_steps, tf.float32)

    def __call__(self, step):
        step = tf.cast(step, tf.float32) + 1e-9
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)
        return tf.math.rsqrt(self.embed_dim) * tf.math.minimum(arg1, arg2)

    def get_config(self):
        return {"embed_dim": float(self.embed_dim), "warmup_steps": float(self.warmup_steps)}
# 3. COSTRUZIONE MODELLO
def build_improved_model(vocab_size: int) -> Model:
    inputs = Input(shape=(config.MAX_LENGTH,))
    x = Embedding(vocab_size, config.EMBED_DIM, mask_zero=False)(inputs)
    x = DynamicPositionalEncoding(config.EMBED_DIM)(x)
    x = Dropout(config.DROPOUT_RATE)(x)
    
    for _ in range(config.TRANSFORMER_LAYERS):
        x = ImprovedTransformerBlock(
            config.EMBED_DIM, 
            config.TRANSFORMER_HEADS, 
            config.FF_DIM, 
            rate=config.DROPOUT_RATE
        )(x)
        
    outputs = Dense(vocab_size)(x)
    
    opt = tf.keras.optimizers.Adam(
        learning_rate=CustomSchedule(config.EMBED_DIM), 
        clipnorm=config.GRADIENT_CLIP
    )
    
    model = Model(inputs, outputs)
    model.compile(optimizer=opt, loss=smoothed_loss)
    return model

# ====================================================================
# [ 8. MONITORAGGIO ]
# ====================================================================

class CustomTensorBoard(tf.keras.callbacks.TensorBoard):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}; lr = self.model.optimizer.learning_rate
        logs['lr'] = lr(epoch).numpy() if isinstance(lr, tf.keras.optimizers.schedules.LearningRateSchedule) else lr.numpy()
        super().on_epoch_end(epoch, logs)

class EnhancedTrainingMonitor(Callback):
    def __init__(self, val_gen: CurriculumSmilesGenerator):
        super().__init__()
        self.val_gen = val_gen
        self.best_val_loss = np.inf; self.prev_val_loss = None

    def generate_sample(self, num: int):
        generated, valid = [], []
        PAD, START, END = [self.val_gen.char2idx[k] for k in ['<PAD>','<START>','<END>']]
        input_seq = np.full((1, config.MAX_LENGTH), PAD, dtype=np.int32)
        
        for _ in range(num):
            input_seq.fill(PAD); input_seq[0, 0] = START
            for t in range(1, config.MAX_LENGTH):
                logits = self.model(input_seq, training=False)[0, t-1]
                probs = tf.nn.softmax(logits / config.TEMPERATURE).numpy()
                if np.sum(probs) < 1e-6: break
                sampled = np.random.choice(len(probs), p=probs)
                input_seq[0, t] = sampled
                if sampled == END: break
            
            indices = input_seq[0].tolist()
            raw_tokens = [self.val_gen.idx2char[i] for i in indices if i not in {PAD, START}]
            
            # Parsing: SMARTS [SEP] TARGET (una sola occorrenza di SEP)
            try:
                sep_idx = raw_tokens.index('<SEP>')
                target_tokens = raw_tokens[sep_idx + 1:]
                smi_str = "".join(target_tokens).split('<END>')[0]
            except (ValueError, IndexError):
                smi_str = "".join(raw_tokens).split('<END>')[0]
            
            final = validate_and_fix_smiles(smi_str)
            if final: valid.append(final)
            generated.append(final or smi_str)
            
        return generated, valid

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        if self.prev_val_loss and 'val_loss' in logs:
            diff = (self.prev_val_loss - logs['val_loss']) / self.prev_val_loss
            self.val_gen.update_complexity(epoch, diff)
        if logs.get('val_loss', np.inf) < self.best_val_loss: self.best_val_loss = logs['val_loss']
        self.prev_val_loss = logs.get('val_loss')
        
        if (epoch + 1) % config.PRINT_EVERY == 0:
            gen, val = self.generate_sample(config.GEN_NUM)
            validity = len(val) / config.GEN_NUM
            novel = len([s for s in val if s not in self.val_gen.train_smiles])
            logger.info(f"\nEPOCH {epoch+1}: Validity {validity:.1%} | Novelty {novel}/{len(val)}")
            if val: logger.info(f"Sample: {val[0]}")

2026-02-16 12:20:04.478642: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-16 12:20:04.492613: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771240804.509778 1263289 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771240804.515122 1263289 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771240804.528501 1263289 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [4]:
# ====================================================================
# [ 9. MAIN EXECUTION ]
# ====================================================================

if __name__ == "__main__":
    logger.info("🚀 START TRAINING: SMARTS-RX → SMILES")
    
    # 1. Verifica Files
    if not os.path.exists(config.SMILES_FILE):
        logger.error("❌ File SMILES non trovato."); sys.exit(1)
    if not os.path.exists(config.SMARTS_RX_FILE):
        logger.error("❌ File SMARTS JSON non trovato."); sys.exit(1)

    with open(config.SMILES_FILE) as f:
        raw_smiles = [line.strip() for line in f if line.strip()]
    logger.info(f"✓ Caricati {len(raw_smiles)} SMILES.")

    # 2. Preprocessing
    processed, vocab, max_len = process_dataset(raw_smiles)
    config.MAX_LENGTH = max_len
    logger.info(f"✓ Vocab: {len(vocab)} token | MaxLen: {max_len}")

    # 3. Salvataggi dizionari
    with open("vocab.json", "w") as f: json.dump(vocab, f)
    with open("char2idx.pkl", "wb") as f: pickle.dump({c:i for i,c in enumerate(vocab)}, f)
    with open("idx2char.pkl", "wb") as f: pickle.dump({i:c for i,c in enumerate(vocab)}, f)
   
    # 4. Split
    stratify = [min(len(t[1]), 20) for t in processed]  # t[1] = target
    try:
        train_data, val_data = train_test_split(processed, test_size=config.VALID_RATIO, stratify=stratify, random_state=42)
    except:
        train_data, val_data = train_test_split(processed, test_size=config.VALID_RATIO, random_state=42)

    # 5. Esempio di Log
    if train_data:
        s_smart, s_targ = train_data[0]
        logger.info(f"\n--- ESEMPIO SEQ ---")
        logger.info(f"SMARTS:   {s_smart}")
        logger.info(f"TARGET:   {''.join(s_targ)}")
        logger.info("-------------------")

    # 6. Avvio training
    train_gen = CurriculumSmilesGenerator(train_data, vocab)
    val_gen = CurriculumSmilesGenerator(val_data, vocab)
     # 1. Salva train_data
    with open("/home/grad/Desktop/pietro/denovo/2/smartrx/train_data.pkl", "wb") as f:
        pickle.dump(train_data, f)
    print(f"✓ train_data salvato: {len(train_data)} molecole")
# 2. Salva val_data
    with open("/home/grad/Desktop/pietro/denovo/2/smartrx/val_data.pkl", "wb") as f:
        pickle.dump(val_data, f)
    print(f"✓ val_data salvato: {len(val_data)} molecole")
    model = build_improved_model(len(vocab))
    model.summary()
    
    callbacks = [
        CustomTensorBoard(log_dir=f"logs/smarts2smiles_{int(time.time())}"),
        EnhancedTrainingMonitor(val_gen),
        ModelCheckpoint("best_smarts2smiles_model.keras", monitor="val_loss", save_best_only=True)
    ]

    try:
        model.fit(
            train_gen.get_dataset(),
            epochs=config.EPOCHS,
            steps_per_epoch=config.STEPS_PER_EPOCH,
            validation_data=val_gen.get_dataset(),
            validation_steps=max(1, len(val_data)//config.BATCH_SIZE),
            callbacks=callbacks
        )
        model.save("final_smarts2smiles_model.keras")
        logger.info("✅ Training Completato.")
    except KeyboardInterrupt:
        logger.warning("⚠️  Training interrotto.")
        model.save("interrupted_smarts2smiles.keras")



2026-02-16 12:19:01,607 [INFO] 🚀 START TRAINING: SMARTS-RX → SMILES
2026-02-16 12:19:01,810 [INFO] ✓ Caricati 891004 SMILES.
2026-02-16 12:19:01,862 [INFO] ✓ Catalogo SMARTS-RX caricato: 406 regole.
2026-02-16 12:19:01,863 [INFO] → Inizio Preprocessing (SMARTS-RX + SMILES)...
2026-02-16 12:19:01,863 [INFO]   Processati 0/891004
[12:19:01] SMILES Parse Error: syntax error while parsing: SMILES
[12:19:01] SMILES Parse Error: Failed parsing SMILES 'SMILES' for input: 'SMILES'


KeyboardInterrupt: 

In [2]:
if __name__ == "__main__":
    logger.info("🚀 AVVIO SCRIPT: CARICAMENTO DIRETTO E ADDESTRAMENTO")
    
    # 1. CONTROLLO FILE ESISTENTI
    if not (os.path.exists(config.TRAIN_DATA_PATH) and 
            os.path.exists(config.VAL_DATA_PATH) and 
            os.path.exists(config.VOCAB_PATH)):
        logger.error(f"❌ File di cache mancanti! Esegui prima lo script di salvataggio.")
        logger.error(f"   Cercati: {config.TRAIN_DATA_PATH}, {config.VAL_DATA_PATH}, {config.VOCAB_PATH}")
        sys.exit(1)

    # 2. CARICAMENTO DATI
    logger.info("📂 Caricamento dati da disco (Pickle)...")
    try:
        with open(config.TRAIN_DATA_PATH, "rb") as f: train_data = pickle.load(f)
        with open(config.VAL_DATA_PATH, "rb") as f: val_data = pickle.load(f)
        with open(config.VOCAB_PATH, "r") as f: vocab = json.load(f)
        logger.info(f"✅ Dati caricati: {len(train_data)} Train, {len(val_data)} Val, {len(vocab)} Vocab")
    except Exception as e:
        logger.error(f"❌ Errore durante il caricamento: {e}")
        sys.exit(1)

    # 3. SETUP GENERATORI
    logger.info("⚙️ Inizializzazione Generatori...")
    train_gen = CurriculumSmilesGenerator(train_data, vocab)
    val_gen = CurriculumSmilesGenerator(val_data, vocab)

    # 4. SETUP MODELLO
    logger.info("🏗️ Costruzione Modello...")
    model = build_improved_model(len(vocab))
    model.summary()

    # 5. AVVIO TRAINING
    logger.info("🔥 Inizio Training Loop...")
    
    callbacks = [
        CustomTensorBoard(log_dir=f"logs/run_loaded_{int(time.time())}"),
        EnhancedTrainingMonitor(val_gen),
        ModelCheckpoint("best_hybrid_model.keras", monitor="val_loss", save_best_only=True)
    ]
    try:
        model.fit(
            train_gen.get_dataset(),
            epochs=config.EPOCHS,
            steps_per_epoch=config.STEPS_PER_EPOCH,
            validation_data=val_gen.get_dataset(),
            validation_steps=max(1, len(val_data)//config.BATCH_SIZE),
            callbacks=callbacks
        )
        model.save("final_hybrid_model.keras")
        logger.info("🏆 Training Completato.")
    except KeyboardInterrupt:
        logger.warning("⚠️ Training interrotto dall'utente.")
        model.save("interrupted_hybrid.keras")

2026-01-12 15:33:24,845 [INFO] 🚀 AVVIO SCRIPT: CARICAMENTO DIRETTO E ADDESTRAMENTO
2026-01-12 15:33:24,846 [INFO] 📂 Caricamento dati da disco (Pickle)...
2026-01-12 15:33:29,620 [INFO] ✅ Dati caricati: 801613 Train, 89069 Val, 349 Vocab
2026-01-12 15:33:29,621 [INFO] ⚙️ Inizializzazione Generatori...
2026-01-12 15:35:57,340 [INFO] 🏗️ Costruzione Modello...
I0000 00:00:1768228557.449495   98451 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9090 MB memory:  -> device: 0, name: NVIDIA RTX A2000 12GB, pci bus id: 0000:65:00.0, compute capability: 8.6


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 100, 512)       │       178,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dynamic_positional_encoding     │ (None, 100, 512)       │             0 │
│ (DynamicPositionalEncoding)     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100, 512)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ improved_transformer_block      │ (None, 100, 512)       │     8,402,944 │
│ (ImprovedTransformerBlock)      │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ improved_transformer_block_1    │ (None, 100, 512)       │     8,402,944 │
│ (ImprovedTransformerBlock)      │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ improved_transformer_block_2    │ (None, 100, 512)       │     8,402,944 │
│ (ImprovedTransformerBlock)      │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ improved_transformer_block_3    │ (None, 100, 512)       │     8,402,944 │
│ (ImprovedTransformerBlock)      │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ improved_transformer_block_4    │ (None, 100, 512)       │     8,402,944 │
│ (ImprovedTransformerBlock)      │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ improved_transformer_block_5    │ (None, 100, 512)       │     8,402,944 │
│ (ImprovedTransformerBlock)      │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 100, 349)       │       179,037 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,775,389 (193.69 MB)

 Trainable params: 50,775,389 (193.69 MB)

...

1000/1000 ━━━━━━━━━━━━━━━━━━━━ 537s 537ms/step - loss: 0.6699 - val_loss: 0.6520 - lr: 9.8821e-05
Epoch 201/400
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 495s 495ms/step - loss: 0.6691 - val_loss: 0.6525 - lr: 9.8575e-05
Epoch 202/400
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 499s 500ms/step - loss: 0.6678 - val_loss: 0.6513 - lr: 9.8331e-05
Epoch 203/400
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 496s 496ms/step - loss: 0.6685 - val_loss: 0.6536 - lr: 9.8088e-05
Epoch 204/400
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 496s 496ms/step - loss: 0.6695 - val_loss: 0.6517 - lr: 9.7848e-05
Epoch 205/400
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 499s 499ms/step - loss: 0.6702 - val_loss: 0.6500 - lr: 9.7609e-05
Epoch 206/400
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 496s 496ms/step - loss: 0.6681 - val_loss: 0.6507 - lr: 9.7371e-05
Epoch 207/400
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 496s 496ms/step - loss: 0.6671 - val_loss: 0.6520 - lr: 9.7136e-05
Epoch 208/400
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 496s 496ms/step - loss: 0.6669 - val_loss: 0.6505 - lr: 9.6902e-05
Epoch 

2026-01-13 22:32:44.564722: W tensorflow/core/framework/op_kernel.cc:1844] UNKNOWN: KeyError: '9'
Traceback (most recent call last):

  File "/home/grad/anaconda3/envs/smiles-transformer/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 269, in __call__
    ret = func(*args)

  File "/home/grad/anaconda3/envs/smiles-transformer/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/home/grad/anaconda3/envs/smiles-transformer/lib/python3.10/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/tmp/ipykernel_98451/3079994727.py", line 271, in __next__
    with self.lock: return next(self.iterator)

  File "/tmp/ipykernel_98451/3079994727.py", line 403, in __call__
    indices = [self.char2idx[t] for t in padded_seq]

  File "/tmp/ipykernel_98451/3079994727.py", line 403, in <li

 770/1000 ━━━━━━━━━━━━━━━━━━━━ 1:25 371ms/step - loss: 0.6644

2026-01-13 22:32:44.930616: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: UNKNOWN: KeyError: '9'
Traceback (most recent call last):

  File "/home/grad/anaconda3/envs/smiles-transformer/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 269, in __call__
    ret = func(*args)

  File "/home/grad/anaconda3/envs/smiles-transformer/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/home/grad/anaconda3/envs/smiles-transformer/lib/python3.10/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/tmp/ipykernel_98451/3079994727.py", line 271, in __next__
    with self.lock: return next(self.iterator)

  File "/tmp/ipykernel_98451/3079994727.py", line 403, in __call__
    indices = [self.char2idx[t] for t in padded_seq]

  File "/tmp/

UnknownError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
2 root error(s) found.
  (0) UNKNOWN:  KeyError: '9'
Traceback (most recent call last):

  File "/home/grad/anaconda3/envs/smiles-transformer/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 269, in __call__
    ret = func(*args)

  File "/home/grad/anaconda3/envs/smiles-transformer/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/home/grad/anaconda3/envs/smiles-transformer/lib/python3.10/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/tmp/ipykernel_98451/3079994727.py", line 271, in __next__
    with self.lock: return next(self.iterator)

  File "/tmp/ip